# EDA

- RMSE : 26185.4795

In [ ]:
!pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
# 열 다 보이게 설정
pd.set_option('display.max_columns', None)
# 행 다 보이게 설정
pd.set_option('display.max_rows', None)

In [ ]:
# 데이터 로드

train_path = '/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/train.csv'
test_path  = '/root/AI_STAGE/upstageailab-ml-competition-ml-2/1.Data/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
# train/test 구분을 위한 칼럼(data)
# train과 test를 하나의 데이터로 만들기

train['data'] = 0
test['data'] = 1
concat = pd.concat([train, test])

print(concat.shape)
print(concat['data'].value_counts())

In [ ]:
# 열 전체를 넣고 스캔하기
train.info()
for col in concat.columns:
    nunique = concat[col].nunique(dropna=False)
    missing_ratio = concat[col].isna().mean()
    missing_count = concat[col].isnull().sum()
    col_type = concat.dtypes[col]
    print(f"📌 {col:30} | 데이터타입: {col_type} | 고유값: {nunique:6} | 결측개수: {missing_count} | 결측률: {missing_ratio:.2%}")

In [ ]:
# 결측치는 아닌데 의미 없는 형식적 값 찾기

def detect_fake_nulls(df, suspect_values=['-', ' ', '', '.', '없음', 'nan']):
    result = {}
    for col in df.columns:
        if concat[col].dtype == 'object':
            val_counts = concat[col].value_counts(dropna=False)
            found = val_counts[val_counts.index.isin(suspect_values)]
            if not found.empty:
                result[col] = found
    return result

fake_nulls = detect_fake_nulls(concat)
for col, vals in fake_nulls.items():
    print(f"🔎 {col} 컬럼에서 의미 없는 값 발견:")
    print(vals)
    print()

In [ ]:
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
concat['도로명'] = concat['도로명'].replace(' ', np.nan)
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)
concat['k-시행사'] = concat['k-시행사'].replace('.', np.nan)
concat['k-시행사'] = concat['k-시행사'].replace('-', np.nan)
concat['k-홈페이지'] = concat['k-홈페이지'].replace('없음', np.nan)
concat['k-홈페이지'] = concat['k-홈페이지'].replace('.', np.nan)

In [ ]:
# 변수별 결측치의 비율을 plot으로 그려보면 아래와 같습니다.
fig = plt.figure(figsize=(13, 2))
missing = concat.isnull().sum() / concat.shape[0]
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(color='orange')
plt.title('변수별 결측치 비율')
plt.show()

In [ ]:
# 결측치 90만개 이상인 값과 이하지만 필요없는 것 제외
# 필요없어 보이는 것 : k-전화번호, k-팩스번호, 사용허가여부, 관리비 업로드, k-수정일자

valid_cols = concat.columns[concat.isnull().sum() <= 900000]
exclude_cols = ['k-전화번호', 'k-팩스번호', '사용허가여부', '관리비 업로드', 'k-수정일자']

select = [col for col in valid_cols if col not in exclude_cols]
concat_select = concat[select]

concat.shape, concat_select.shape

In [ ]:
def null_summary(df, columns):
    result = pd.DataFrame({
        '결측 개수': df[columns].isnull().sum(),
        '결측 비율(%)': df[columns].isnull().mean() * 100
    })
    return result[result['결측 개수'] > 0].sort_values('결측 비율(%)', ascending=False)

print("📊 연속형 변수 결측치 요약")
display(null_summary(concat_select, con_columns))

print("📊 범주형 변수 결측치 요약")
display(null_summary(concat_select, cat_columns))


In [ ]:
# 결측치 탐지 및 처리 전 연속형 변수 상관관계 보기

# 1. 수치형 변수만 선택
numeric_cols = concat_select.select_dtypes(include=['float64', 'int64']).columns

# 2. 상관관계 계산
corr = concat_select[numeric_cols].corr()

# 3. 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("📊 수치형 변수 간 상관관계")
plt.show()

In [ ]:
#상관관계 기반으로 중복 feature 쌍 탐지 및 삭제 후보 추천

# 연속형 변수만 추출
numeric_cols = con_columns  # ← 너가 나눠둔 연속형 변수 리스트

# 상관관계 행렬 (절댓값 기준)
corr_matrix = concat_select[numeric_cols].corr().abs()

# 상삼각 행렬로 중복 제거
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 상관계수 0.9 초과인 변수쌍 추출
high_corr_pairs = [(col, row, upper.loc[row, col])
                   for col in upper.columns
                   for row in upper.index
                   if pd.notnull(upper.loc[row, col]) and upper.loc[row, col] > 0.7]

# 출력
for col1, col2, score in sorted(high_corr_pairs, key=lambda x: -x[2]):
    print(f"🔁 {col1} ↔ {col2} : 상관계수 = {score:.2f}")


In [ ]:
# 전용세대별면적 PCA 진행 및 상관관계 높은 변수 제거 후 데이터 살펴보기

concat_select.head(1)

In [ ]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
# 숫자형 분리 pd.api.types.is_numeric_dtype
con_columns2 = []
cat_columns2 = []

for column in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[column]):
        con_columns2.append(column)
    else:
        cat_columns2.append(column)

print("연속형 변수:", con_columns2)
print("범주형 변수:", cat_columns2)

def null_summary(df, columns):
    result = pd.DataFrame({
        '결측 개수': df[columns].isnull().sum(),
        '결측 비율(%)': df[columns].isnull().mean() * 100
    })
    return result[result['결측 개수'] > 0].sort_values('결측 비율(%)', ascending=False)

print("📊 연속형 변수 결측치 요약")
display(null_summary(concat_select, con_columns2))

print("📊 범주형 변수 결측치 요약")
display(null_summary(concat_select, cat_columns2))


In [ ]:
# 범주형 feature들 관계 보기

from scipy.stats import chi2_contingency

# Cramér's V 계산 함수
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

# 범주형 변수 리스트
cat_cols = cat_columns2  # 이미 나눈 리스트

# 결과 저장용 매트릭스
cramer_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols)

for col1 in cat_cols:
    for col2 in cat_cols:
        if col1 == col2:
            cramer_matrix.loc[col1, col2] = 1.0
        else:
            try:
                cramer_matrix.loc[col1, col2] = cramers_v(concat_select[col1], concat_select[col2])
            except:
                cramer_matrix.loc[col1, col2] = np.nan

plt.figure(figsize=(14, 12))
sns.heatmap(cramer_matrix.astype(float), cmap='coolwarm', annot=False)
plt.title("범주형 변수 간 Cramér's V (상관관계 유사도)")
plt.show()


In [ ]:
# 변주형 중 unique값이 낮은 것
# 고유값이 적으면 다른 변수와 관계가 높을 수 있다고 함

for col in cat_columns2:
    print(f"{col}: {concat_select[col].nunique()}")


In [ ]:
threshold = 0.8
high_corr_pairs = []

for i in range(len(cat_cols)):
    for j in range(i + 1, len(cat_cols)):
        col1, col2 = cat_cols[i], cat_cols[j]
        val = cramer_matrix.loc[col1, col2]
        if pd.notnull(val) and float(val) >= threshold:
            high_corr_pairs.append((col1, col2, float(val)))

for col1, col2, score in sorted(high_corr_pairs, key=lambda x: -x[2]):
    print(f"🔁 {col1} ↔ {col2} : Cramér's V = {score:.2f}")


In [ ]:
# 이상치 탐색

def detect_outliers_iqr(df, columns, iqr_scale=1.5):
    outlier_summary = []

    for col in columns:
        if df[col].isnull().all():
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_scale * IQR
        upper_bound = Q3 + iqr_scale * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = outliers.shape[0]
        outlier_ratio = outlier_count / df.shape[0] * 100

        outlier_summary.append({
            '변수': col,
            '이상치 개수': outlier_count,
            '이상치 비율(%)': round(outlier_ratio, 2)
        })

    return pd.DataFrame(outlier_summary).sort_values('이상치 비율(%)', ascending=False)


In [ ]:
# 이상치 비율 상위 5개 변수 추출
top_outlier_cols = outlier_df['변수'].head(5)

# Boxplot 시각화
plt.figure(figsize=(15, 8))
for i, col in enumerate(top_outlier_cols, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x=train_data[col])
    plt.title(col)

plt.tight_layout()
plt.show()
